# Dictionaries and sets

Python dicts are highly optimized.

Dictionaries are used in:


## Module namespaces

In [2]:
#import a module
import argparse
#access the modules namespace
args = argparse.Namespace()
#insert some dict elements
args.foo = 1
args.bar = [1,2,3]
#print the dictionary, using the vars() method
print(vars(args))

{'foo': 1, 'bar': [1, 2, 3]}


## Class and instance attributes

In [8]:
#define a new class
class new_class():
    def __init__(self, number):
        self.multi = int(number) * 2
        self.str = str(number)
#instantiate class
a = new_class(2)
#print object as dict
print(a.__dict__)

{'multi': 4, 'str': '2'}


## Function keyword arguments

In [ ]:
#define a function that takes both positional and keyword arguments
def my_function(*positional,**kwargs):
    print("Positional:", positional, "as a tuple")
    print("Keywords:", kwargs, "as a dictionary")
#pass positional arguments, and keyword args.
my_function('one', 'two', 'three', a=12, b="abc")

There are lots of ways to make dictionaries

In [16]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3])) 
d = dict([('two', 2), ('one', 1), ('three', 3)]) 
e = dict({'three': 3, 'one': 1, 'two': 2}) 
a==b==c==d==e

True

## Dict comprehension in Python >2.7 (like listcomp)


In [18]:
DIAL_CODES = [(86, 'China'),(91, 'India'),(1, 'United States'),(62, 'Indonesia'),(55, 'Brazil')]
#Use the dict comp syntax - i.e. with {} instead of []
country_code = {country: code for code, country in DIAL_CODES}
print(country_code)

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62, 'Brazil': 55}


In [44]:
import re

#regex to get unique instances of a word
WORD_RE = re.compile('\w+')
#index is a dict
index = {}

#open the text file
with open('zen.txt', encoding='utf-8') as fp:
    #for each line in the file
    for line_no, line in enumerate(fp, 1):
        #use regex to get each word in the line 
        for match in WORD_RE.finditer(line):
            #group / deduplicate words in line
            word = match.group()
            #only words over 9 chars
            if len(word) < 9:
                continue
            #isolate the match column, adding one to get accurate line numbers
            column_no = match.start()+1
            #concatenate line & column for location
            location = (line_no, column_no)
            #this is UGLY; coded like this to make a point
            #Lookup 1: Get the list of occurrences for word in the index dict, or add empty [] if not already in list.
            occurrences = index.get(word, [])
            #Append new location as tuple to occurrence list. May be empty if first occurrence.
            occurrences.append(location)
            #Lookup 2: Overwrite occurrences back to index dict; this entails a second search through the index.
            index[word] = occurrences
            #BETTER: Get list of occurrences, set empty list if not found, then append the occurrence tuple
            #Single lookup using setdefault, rather than get with default
            index.setdefault(word, []).append(location)
            
# print in alphabetical order
for word in sorted(index, key=str.upper): print(word, index[word])

ambiguity [(12, 16), (12, 16)]
Beautiful [(1, 1), (1, 1)]
complicated [(4, 24), (4, 24)]
explicitly [(11, 8), (11, 8)]
implementation [(17, 8), (17, 8), (18, 8), (18, 8)]
Namespaces [(19, 1), (19, 1)]
practicality [(9, 10), (9, 10)]
preferably [(13, 25), (13, 25)]
Readability [(7, 1), (7, 1)]
temptation [(12, 38), (12, 38)]


## Mappings with flexible key lookup

A defaultdict is configured to create items on demand whenever a missing key is searched.

In [46]:
import re
import collections

WORD_RE = re.compile('\w+')
#this time, index is a defaultdict. 
#If key is not present, a list is created, the key inserted, and list reference returned
index = collections.defaultdict(list) #list is the 'list constructor', used as the default_factory

with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1): 
        for match in WORD_RE.finditer(line):
            word = match.group()
            if len(word) < 9:
                continue
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location)
            
# print in alphabetical order
for word in sorted(index, key=str.upper): print(word, index[word])

ambiguity [(12, 16)]
Beautiful [(1, 1)]
complicated [(4, 24)]
explicitly [(11, 8)]
implementation [(17, 8), (18, 8)]
Namespaces [(19, 1)]
practicality [(9, 10)]
preferably [(13, 25)]
Readability [(7, 1)]
temptation [(12, 38)]


## The __missing__ method

Underlying the way mappings deal with missing keys is the aptly named __missing__ method. This method is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a __missing__ method, the standard dict.__getitem__ will call it whenever a key is not found, instead of raising KeyError.

In [52]:
class StrKeyDict0(dict):
    def __missing__(self, key): 
        if isinstance(key, str):
            raise KeyError(key) 
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError: 
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


d = StrKeyDict0([('2', 'two'), ('4', 'four')])
#pass a string - found easily
print(d['2'])
#pass a int, convert to string, find
print(d[4])
#print(d[1]) >>>throws KeyError1, because 1 is an int and isn't found
#pass an string, no problem
print(d.get('2'))
#pass an int, convert & find
print(d.get(4))
#pass an unknown string: pass, convert, return - not added to dict though
print(d.get(1,'N/A'))
print(d)


two
four
two
four
N/A
{'2': 'two', '4': 'four'}


## Variations of dict
-  collections.OrderedDict
    -  maintains keys in insertion order, allowing iteration over items in a predictable order.
-  collections.ChainMap
    -  holds a list of mappings which can be searched as one. eg - named tuple containing named tuples
-  collections.counter
    -  a mapping that holds an integer count for each key. Updating an existing key adds to its count. 
-  collections.UserDict
    -  a pure Python implementation of a mapping that works like a standard dict.
    
## Subclassing UserDict

It’s almost always easier to create a new mapping type by extending UserDict than dict. The main reason why it’s preferable to subclass from UserDict than dict is that the built-in has some implementation shortcuts that end up forcing us to override methods that we can just inherit from UserDict with no problems.

In [61]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key): 
        if isinstance(key, str):
            raise KeyError(key) 
        return self[str(key)]

    def __contains__(self, key): 
        return str(key) in self.data

    def __setitem__(self, key, item):
        print(item)
        self.data[str(key)] = item
    
e = StrKeyDict([('2', 'two'), ('4', 'four')])
#pass a string - found easily
print(e['2'])
#pass a int, convert to string, find
print(e[4])
#print(d[1]) >>>throws KeyError1, because 1 is an int and isn't found
#pass an string, no problem
print(e.get('2'))
#pass an int, convert & find
print(e.__contains__(4))
#pass an unknown string: pass, convert, add, return
print(e.__setitem__(1,'N/A'))
print(e)


two
four
two
four
two
True
N/A
None
{'2': 'two', '4': 'four', '1': 'N/A'}


Mappingproxy instance that is a read-only but dynamic view of the original mapping

In [64]:
from types import MappingProxyType

#make a doct
d = {1:'A'}
#make a proxy of it
d_proxy = MappingProxyType(d)
#print it - no problems
print(d_proxy)
#updated - can't
print(d_proxy[1])
#d_proxy[2] = 'x' # throws TypeError: 'mappingproxy' object does not support item assignment
d[2] = 'B'
print(d_proxy)
print(d_proxy[2])

{1: 'A'}
A
{1: 'A', 2: 'B'}
B


## Set theory

A set is a collection of unique objects. A basic use case is removing duplication. Set elements must be hashable. The set type is not hashable, but frozenset is, so you can have frozenset elements inside a set.

## Set operations

There are lots of manipulation methods, and operators, based on mathematical set theory

In [8]:
needles = {1}
haystack = {2}
#Count needles in haystack, using asuming vars are sets
found = len(needles & haystack)

#Same as above, using for
found = 0
for n in needles:
    if n in haystack: found += 1

#Count needles, building two sets - could be cheaper if one element is already a set
found = len(set(needles) & set(haystack))
#alternative synatx of above
found = len(set(needles).intersection(haystack))

#Set literals
s = {1}
print(s)
print(type(s))
s.pop()
#empty set becomes 
print(s)

#Frozen sets must use the constructor
print(frozenset(range(10)))

#set comprehension, same syntax as list comp
from unicodedata import name
print({chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')})

{1}
<class 'set'>
set()
frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})
{'®', '÷', '$', '£', '×', '¥', '¶', '+', '±', '<', '%', '¤', '=', '°', '¬', '>', 'µ', '§', '©', '#', '¢'}


## Use of Dicts and Sets

These are fast datastructures, because the can make use of hash tables for lookup, because they are immutable. Python has optimisations for speed, and aims to keep the hash load factor at < 70%. When load factor increases, a new hash table is created. Collisions are resolved with linear probing(?). Because hash tables are used:

-  Dict keys must be hashable
-  dicts have significant memory overhead (i.e. empty table needed & field names stored in record.
-  Key search is FAST
-  Key ordering depends on insertion order, as first of identical insertion doesn't have a collision
-  

In [11]:
# dial codes of the top 10 most populous countries
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
]
d1 = dict(DIAL_CODES) 
print('d1:', d1.keys())
d2 = dict(sorted(DIAL_CODES))
print('d2:', d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1])) 
print('d3:', d3.keys())
#The dictionaries compare equal, because they hold the same key:value pairs.
print(d1==d2 and d2==d3)

d1: dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])
d2: dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])
d3: dict_keys([880, 55, 86, 91, 62, 81, 234, 92, 7, 1])
True


## How sets work — practical consequences
The set and frozenset types are also implemented with a hash table, except that each bucket holds only a reference to the element (as if it were a key in a dict, but without a value to go with it). Therefore:

-  Set elements must be hashable objects.
-  Sets have a significant memory overhead.
-  Membership testing is very efficient.
-  Element ordering depends on insertion order.
-  Adding elements to a set may change the order of other elements.

## Summary

Dictionaries are a keystone of Python. Beyond the basic dict, the standard library offers handy, ready-to-use specialized mappings like defaultdict, OrderedDict, ChainMap and Counter, all defined in the collections module.

Two powerful methods available in most mappings are setdefault and update. The setdefault method is used to update items holding mutable values, for example, in a dict of list values, to avoid redundant searches for the same key. The update method allows bulk insertion or overwriting of items from any other mapping, from iterables providing (key, value) pairs and from keyword arguments.

The hash table implementation underlying dict and set is extremely fast. There is a price to pay for all this speed, and the price is in memory.
